In [2]:
%load_ext autoreload
%autoreload 2
from gurobipy import *
import gurobipy as gp
import numpy as np

import ventutils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def printVars(z, string = "", tol = 1e-7):
    for zz in z:
        if abs(z[zz].X) >= tol :
            print(string,zz,"----", z[zz].x)
            
def printParams(z, string = "", tol = 1e-7):
    for zz in z:
        if abs(z[zz]) >= tol :
            print(string,zz,"----", z[zz])

# Data

In [4]:
# Q[i][t]: Demand at location i at time t.
Q = [list(x) for x in zip(*ventutils.ode_data())] # ventutils.ihme_data()
Qarr = np.array(Q)
# np.sum(Qarr, axis=0) # Total demand on each Day

# n: Number of locations.
n = len(Q)

# T: Time horizon.
T = len(Q[0])

# D[i][j]: Delay in transfer from location i to location j.
# TODO: Make this relative to the distance between locations (1000 miles = 1 day?)
D = [[1 for _ in range(n)] for _ in range(n)]

Qarr, Qarr.shape

(array([[50, 51, 52, ..., 11, 11, 10],
        [ 5,  5,  5, ...,  1,  1,  1],
        [65, 67, 68, ..., 13, 12, 12],
        ...,
        [11, 12, 13, ...,  4,  4,  4],
        [59, 60, 61, ..., 13, 12, 12],
        [ 6,  6,  6, ...,  1,  1,  1]]),
 (50, 53))

In [142]:
# temp  = np.round(np.average(Qarr, axis=0))
# Q = [list(temp) for i in range(n)]
# Qarr = np.array(Q)
# Qarr, Qarr.shape

# Parameters

In [143]:
# V[i]: Initial supply of ventilators at location i.
V = ventutils.initial_supply()

# R: Recovery time.
R = 10

# N_max: Number of ventilators available to be injected into the system each day
N_max = np.sum(Qarr, axis = 0)/20 # 65000/T



# Efficiency
Eff = 0.1

temp = sum(V)
V = [ int(vv/10) for vv in V]


# Variables

In [144]:
M = gp.Model()

# x[i, t]: Number of ventilators at location i at time t.
x = M.addVars(n, T, lb=0, vtype=gp.GRB.INTEGER, name='x')

# N[t]: Number of ventilators injected into the system at time t.
N = M.addVars(T, lb=0.9*N_max, ub = N_max, vtype=gp.GRB.INTEGER, name='N')

# z[i, t]: How many of the new ventilators N[t] are sent to location i at time t.
z = M.addVars(n, T, lb=0, vtype=gp.GRB.INTEGER, name='z')

# f[i, j, t]: How many ventilators location i sends to location j at time t.
D_max = max([max(i) for i in D])
f = M.addVars(n, n, range(-D_max, T),
              lb=0, vtype=gp.GRB.INTEGER, name='f')
# Negative indices are required for checking t-D[i][j] when t is small.
# For these indices f is zero.
for t in range(-D_max, 0):
    for i in range(n):
        for j in range(n):
            f[i, j, t].lb = 0
            f[i, j, t].ub = 0
            
# No transfer to the same location!
for t in range(0, T):
    for i in range(n):
        f[i, i, t].lb = 0
        f[i, i, t].ub = 0

# tau[i, t]: Shortage of ventilators at location i at time t.
tau = M.addVars(n, T, lb=-GRB.INFINITY, vtype=gp.GRB.INTEGER, name='tau')
tauPos = M.addVars(n, T, lb=0, vtype=gp.GRB.INTEGER, name='tauPos')

# phi: Objective.
phi = M.addVar(obj=1, lb=0, vtype=gp.GRB.INTEGER, name='phi')
phi_ = M.addVars(n, lb=0, vtype=gp.GRB.INTEGER, name='phi_')



Using license file /home/pholi/Temp/gurobi/gurobi901/gurobi.lic
Academic license - for non-commercial use only


# Constraints

In [145]:
# The number of ventilators in the system at time t should not be greater than
# the combination of the initial supply of ventilators and what has been
# injected into the system since the beginning.
for t in range(T):
    M.addConstr(gp.quicksum(x[i, t] for i in range(n)) <=
                sum(V) + gp.quicksum(N[tp] for tp in range(t+1)))
#     M.addConstr(gp.quicksum(x[i, t] for i in range(n)) >=
#                 Eff*sum(V) + Eff*gp.quicksum(N[tp] for tp in range(t+1)))

# The number of new ventilators assigned at time t should not be greater than
# the number of new ventilators injected into the system at time t.
for t in range(T):
    M.addConstr(gp.quicksum(z[i, t] for i in range(n)) == N[t])

# The total number of ventilators injected into the system must not be greater
# than what is available to be injected.
# M.addConstr(gp.quicksum(N[t] for t in range(T)) <= N_max)
    
# The number of ventilators in the various locations should be consistent over
# time, including consistency with the new ventilators injected into the system
# and with the transfer of ventilators between locations.
for i in range(n):
    for t in range(1, T):
        M.addConstr(x[i, t] ==
                    x[i, t-1] +
                    z[i, t] +
                    gp.quicksum(f[j, i, t-D[j][i]] for j in range(n)) -
                    gp.quicksum(f[i, j, t] for j in range(n)))

# A ventilator cannot be moved before its recovery time.
for i in range(n):
    for t in range(1, T):
        M.addConstr(x[i, t] >=
                    gp.quicksum(z[i, tp]+gp.quicksum(f[j, i, tp-D[j][i]] for j in range(n)) for tp in range(max(1, t-R), t)))

# Keep track of the shortages of ventilators.
for i in range(n):
    for t in range(T):
        # No max needed since LB of tau is already 0
        M.addConstr(tau[i, t] == Q[i][t]-x[i, t])
        M.addGenConstrMax(tauPos[i,t],[tau[i,t]],0)
        

# Objective

In [146]:
for i in range(n):
    M.addConstr(phi_[i] == gp.quicksum(tauPos[i, t] for t in range(T)))

# M.addConstr(phi == gp.quicksum(phi_[i] for i in range(n)))
phiMin = M.addVar(obj=-1, lb=0, name="phiMin")
M.addGenConstrMin(phiMin, phi_)
    
M.addGenConstrMax(phi, phi_)
M.params.Threads =2
M.update()

Changed value of parameter Threads to 2
   Prev: 0  Min: 0  Max: 1024  Default: 0


In [147]:
M.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 8006 rows, 145705 columns and 1470934 nonzeros
Model fingerprint: 0x788f6f15
Model has 2652 general constraints
Variable types: 1 continuous, 145704 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+02, 7e+02]
  RHS range        [1e+00, 7e+03]
Presolve added 7758 rows and 0 columns
Presolve removed 0 rows and 2252 columns
Presolve time: 3.01s
Presolved: 15764 rows, 143453 columns, 1431648 nonzeros
Presolved model has 2741 SOS constraint(s)
Variable types: 0 continuous, 143453 integer (7859 binary)

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.075375e+03   1.578600e+10      5s
     975    1.2729600e+05   0.000000e+00   1.600000e+01      5s
    8755    7.4480584e+03   0

# Greedy Stuff

In [11]:
tStart = 0
tEnd = 5
TT = [tt for tt in range(tStart, tEnd)]
Qcurr = {tt:[Q[i][tt] for i in range(50)] for tt in range(tStart, tEnd)}
T = len(Q[0])
D_max = max([max(i) for i in D])

In [12]:
xpast = {(i,t):0 for (i,t) in itertools.product(range(n),range(0,tStart))}
Npast = {t:0 for t in range(0,tStart)}
zpast = {(i,t):0 for (i,t) in itertools.product(range(n),range(0,tStart))}
fpast = {(i,j,t):0 for (i,j,t) in itertools.product(range(n),range(n),range(0,tStart))}
phi_past = {i:0 for i in range(n)}
# phi_past = {(i,t):0 for (i,t) in itertools.product(range(n),range(0,tStart))}

# taupast = {(i,t):0 for (i,t) in itertools.product(range(n),range(0,tStart))}
# tauPospast = {(i,t):0 for (i,t) in itertools.product(range(n),range(0,tStart))}



In [13]:
period = 5
for time in range(0,T,period):
    tStart = time
    tEnd = min(time+period, T)
    Mgreed, x,N,z, f, phi_ = ventutils.greedyIter(n, tStart, tEnd, V, Q, D, N_max, R, xpast, Npast, zpast, fpast, phi_past)
    for t in range(tStart, tEnd):
        for i in range(n):
            xpast[(i,t)] = x[i,t].X
            zpast[(i,t)] = z[i,t].X
            for j in range(n):
                fpast[(i,j,t)] = f[i,j,t].X
        Npast[t] = N[t].X
    for i in range(n):
        phi_past[i] = phi_[i,tEnd-1].X

Changed value of parameter Threads to 2
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 1010 rows, 13765 columns and 40770 nonzeros
Model fingerprint: 0x349abf41
Model has 6125 SOS constraints
Model has 260 general constraints
Variable types: 0 continuous, 13765 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e+00]
  Bounds range     [5e+02, 6e+02]
  RHS range        [5e+00, 7e+03]
Presolve added 2634 rows and 969 columns
Presolve time: 0.27s
Presolved: 3644 rows, 14734 columns, 45630 nonzeros
Presolved model has 5108 SOS constraint(s)
Variable types: 0 continuous, 14734 integer (1480 binary)

Root relaxation: objective 3.253700e+04, 1460 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 32537.0000    0    4          - 3


Explored 1 nodes (662 simplex iterations) in 0.62 seconds
Thread count was 2 (of 4 available processors)

Solution count 1: 52360 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.236000000000e+04, best bound 5.236000000000e+04, gap 0.0000%
Changed value of parameter Threads to 2
   Prev: 0  Min: 0  Max: 1024  Default: 0
Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (linux64)
Optimize a model with 1010 rows, 13765 columns and 40970 nonzeros
Model fingerprint: 0x9f6913b9
Model has 6125 SOS constraints
Model has 260 general constraints
Variable types: 0 continuous, 13765 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-03, 1e+00]
  Bounds range     [4e+02, 5e+02]
  RHS range        [2e+00, 2e+04]
Presolve added 1667 rows and 497 columns
Presolve time: 0.24s
Presolved: 2677 rows, 14262 columns, 43378 nonzeros
Presolved model has 5444 SOS constraint(s)
Variable types: 0 continuous, 14262 integer (1079 binary)

Root relaxatio

In [14]:
printParams(xpast,"x")

x (31, 0) ---- 512.0
x (31, 1) ---- 1049.0
x (31, 2) ---- 1609.0
x (31, 3) ---- 2190.0
x (31, 4) ---- 2790.0
x (29, 5) ---- 92.0
x (31, 5) ---- 3315.0
x (29, 6) ---- 724.0
x (31, 6) ---- 3315.0
x (29, 7) ---- 1368.0
x (31, 7) ---- 3315.0
x (29, 8) ---- 1736.0
x (31, 8) ---- 3600.0
x (1, 9) ---- 285.0
x (10, 9) ---- 9.0
x (29, 9) ---- 1911.0
x (31, 9) ---- 3726.0
x (0, 10) ---- 202.0
x (1, 10) ---- 285.0
x (10, 10) ---- 9.0
x (16, 10) ---- 26.0
x (18, 10) ---- 13.0
x (23, 10) ---- 59.0
x (25, 10) ---- 8.0
x (27, 10) ---- 49.0
x (29, 10) ---- 2000.0
x (31, 10) ---- 3726.0
x (33, 10) ---- 15.0
x (38, 10) ---- 36.0
x (40, 10) ---- 13.0
x (44, 10) ---- 10.0
x (47, 10) ---- 15.0
x (48, 10) ---- 60.0
x (49, 10) ---- 5.0
x (0, 11) ---- 805.0
x (1, 11) ---- 285.0
x (10, 11) ---- 9.0
x (16, 11) ---- 26.0
x (18, 11) ---- 13.0
x (23, 11) ---- 59.0
x (25, 11) ---- 8.0
x (27, 11) ---- 49.0
x (29, 11) ---- 2000.0
x (31, 11) ---- 3726.0
x (33, 11) ---- 15.0
x (38, 11) ---- 36.0
x (40, 11) ---- 13.0
x 

x (28, 26) ---- 23.0
x (29, 26) ---- 2589.0
x (30, 26) ---- 15.0
x (31, 26) ---- 3726.0
x (33, 26) ---- 22.0
x (35, 26) ---- 44.0
x (36, 26) ---- 25.0
x (37, 26) ---- 1009.0
x (38, 26) ---- 36.0
x (40, 26) ---- 13.0
x (42, 26) ---- 84.0
x (43, 26) ---- 60.0
x (44, 26) ---- 10.0
x (46, 26) ---- 80.0
x (47, 26) ---- 589.0
x (48, 26) ---- 60.0
x (49, 26) ---- 2064.0
x (0, 27) ---- 2202.0
x (1, 27) ---- 3.0
x (3, 27) ---- 11.0
x (4, 27) ---- 541.0
x (5, 27) ---- 69.0
x (7, 27) ---- 32.0
x (8, 27) ---- 599.0
x (10, 27) ---- 9.0
x (11, 27) ---- 58.0
x (12, 27) ---- 285.0
x (14, 27) ---- 26.0
x (15, 27) ---- 15.0
x (16, 27) ---- 26.0
x (17, 27) ---- 646.0
x (18, 27) ---- 13.0
x (20, 27) ---- 260.0
x (21, 27) ---- 653.0
x (23, 27) ---- 61.0
x (25, 27) ---- 8.0
x (26, 27) ---- 10.0
x (27, 27) ---- 49.0
x (28, 27) ---- 23.0
x (29, 27) ---- 2589.0
x (30, 27) ---- 15.0
x (31, 27) ---- 3726.0
x (33, 27) ---- 22.0
x (35, 27) ---- 44.0
x (36, 27) ---- 25.0
x (37, 27) ---- 1009.0
x (38, 27) ---- 36.0


x (11, 37) ---- 58.0
x (12, 37) ---- 285.0
x (14, 37) ---- 26.0
x (15, 37) ---- 15.0
x (16, 37) ---- 26.0
x (17, 37) ---- 1074.0
x (18, 37) ---- 13.0
x (20, 37) ---- 260.0
x (21, 37) ---- 653.0
x (22, 37) ---- 6.0
x (23, 37) ---- 61.0
x (25, 37) ---- 8.0
x (26, 37) ---- 10.0
x (27, 37) ---- 49.0
x (28, 37) ---- 23.0
x (29, 37) ---- 3026.0
x (30, 37) ---- 15.0
x (31, 37) ---- 3726.0
x (32, 37) ---- 58.0
x (33, 37) ---- 26.0
x (34, 37) ---- 52.0
x (35, 37) ---- 44.0
x (36, 37) ---- 25.0
x (37, 37) ---- 1009.0
x (38, 37) ---- 36.0
x (40, 37) ---- 13.0
x (42, 37) ---- 353.0
x (43, 37) ---- 60.0
x (44, 37) ---- 10.0
x (46, 37) ---- 80.0
x (47, 37) ---- 589.0
x (48, 37) ---- 60.0
x (49, 37) ---- 3565.0
x (0, 38) ---- 3308.0
x (1, 38) ---- 3.0
x (2, 38) ---- 37.0
x (3, 38) ---- 11.0
x (4, 38) ---- 541.0
x (5, 38) ---- 69.0
x (6, 38) ---- 154.0
x (7, 38) ---- 32.0
x (8, 38) ---- 599.0
x (10, 38) ---- 9.0
x (11, 38) ---- 58.0
x (12, 38) ---- 285.0
x (14, 38) ---- 26.0
x (15, 38) ---- 15.0
x (16

In [15]:
printParams(zpast, "z")

z (31, 0) ---- 512.0
z (31, 1) ---- 537.0
z (31, 2) ---- 560.0
z (31, 3) ---- 581.0
z (31, 4) ---- 600.0
z (29, 5) ---- 92.0
z (31, 5) ---- 525.0
z (29, 6) ---- 632.0
z (29, 7) ---- 644.0
z (29, 8) ---- 368.0
z (31, 8) ---- 285.0
z (1, 9) ---- 285.0
z (10, 9) ---- 9.0
z (29, 9) ---- 175.0
z (31, 9) ---- 126.0
z (0, 10) ---- 202.0
z (16, 10) ---- 26.0
z (18, 10) ---- 13.0
z (23, 10) ---- 59.0
z (25, 10) ---- 8.0
z (27, 10) ---- 49.0
z (29, 10) ---- 89.0
z (33, 10) ---- 15.0
z (38, 10) ---- 36.0
z (40, 10) ---- 13.0
z (44, 10) ---- 10.0
z (47, 10) ---- 15.0
z (48, 10) ---- 60.0
z (49, 10) ---- 5.0
z (0, 11) ---- 603.0
z (14, 12) ---- 26.0
z (17, 12) ---- 577.0
z (3, 13) ---- 11.0
z (7, 13) ---- 32.0
z (11, 13) ---- 58.0
z (23, 13) ---- 2.0
z (29, 13) ---- 24.0
z (37, 13) ---- 475.0
z (8, 14) ---- 599.0
z (15, 15) ---- 15.0
z (26, 15) ---- 10.0
z (28, 15) ---- 23.0
z (29, 15) ---- 2.0
z (30, 15) ---- 15.0
z (33, 15) ---- 4.0
z (35, 15) ---- 44.0
z (36, 15) ---- 25.0
z (37, 15) ---- 522.0
